# Generating Text with LSTM Networks

Below you see the overview of steps that I took. Notebook is a bit long so i thought it would be easier to navigate if I put an overview here.
1. Predict less than 1000 characters: Resulted in better things with 300. Also 500 is promising.
2. Clean the source code with punction: Resulted with repeating patterns a lot. Due to this decided to test with another resourse
3. The adventures of pinocchio book is downloaded and tested: Result was still repating itself and decided to increase epochs to train more
4. Epochs are increase from 20 to 40: Resulted in a good way. That showed me 2 and 3 steps are bad because of high loss and lack of enough epochs. I expected to this perfroms good also because of losses.
5. Increasing Dropout to 0.3: This lead to high losses because we randamly drop more stuff. Trained with 40 epoch but I think its not enough (This was expected. ). It again repeats itself training with more needed. It starts good but make repatation with 5-6 word sequences. Lets play with batch size a bit more.
6. Change of batch size: This will make even harder to train. Test made with 32 batch. Which resulted longer train. Result is bad. Losses didnt even improved, I stopped in 5 epoch and tested with 64 batch again to compare. Conclusion is low batch size with higher dropout is didnt worked with this model I think thi is because it drops randomly and trains for more generaly with batches resulting in issues in training. Basicly its not able to train. I wasnt expecting to be like this. It was worse.
7. More layers: I think its unnecesery because its already a complex model. But wanted to experiment so added one more layer. Result is so bad. I even think if I missed sth and it become soo bad. Also attached the checkpoint file so you can test.

So all these 7 steps can be found below.

After running all my final conclusion is, having enough training epochs are important at least 50's was needed for me to have good results. High dropot and low batch combination is not a good choice dur to lack of improvement in loss. Decrease of output length generally performs better. Instead removing all puntions, leaving important punctions performs better.





In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [2]:
# load ascii text and covert to lowercase
filename = "wonderland_text.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: {}".format(n_chars))
print("Total Vocab: {}".format(n_vocab))
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: {}".format(n_patterns))
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalise the inputs to be in the range 0..1
X = X / float(n_vocab)
# one hot encode the outputs
y = to_categorical(dataY)


Total Characters: 144572
Total Vocab: 49
Total Patterns: 144472


In [4]:
# define the LSTM model
bigger_model = Sequential()
bigger_model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
bigger_model.add(Dropout(0.2))
bigger_model.add(LSTM(256, return_sequences=True))
bigger_model.add(Dropout(0.2))
bigger_model.add(LSTM(256))
bigger_model.add(Dropout(0.2))
bigger_model.add(Dense(y.shape[1], activation='softmax'))
bigger_model.compile(loss='categorical_crossentropy', optimizer='adam')

bigger_model.summary()
bigger_filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.keras"
bigger_checkpoint = ModelCheckpoint(bigger_filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
bigger_callbacks_list = [bigger_checkpoint]
# load the network weights
int_to_char = dict((i, c) for i, c in enumerate(chars))

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 100, 256)            │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 100, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 100, 256)            │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 100, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 256)                 │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 49)                  │          12,593 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,327,409 (5.06 MB)

 Trainable params: 1,327,409 (5.06 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# fit the model
bigger_model.fit(X, y, epochs=50, batch_size=64, callbacks=bigger_callbacks_list)

Epoch 1/50
1226/2258 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - loss: 3.1008

KeyboardInterrupt: 

In [6]:
# load the network weights, i put this in submisison
bigger_filename = "weights-improvement-50-1.0725-bigger.keras"
bigger_model.load_weights(bigger_filename)
bigger_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [6]:
import sys

# pick a random seed
def generate_text(model, length=1000):
  start = np.random.randint(0, len(dataX)-1)
  pattern = dataX[start]
  output = ""
  print("Seed:")
  print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
  print("\n\nGenerated:")
  # generate characters
  for i in range(length):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    output += result
    sys.stdout.write(result)
    sys.stdout.flush()
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
  print("\n\nDone")

Until know everything is same with the given ipynb. Now starting to experiment
# 1. Use less than 1000 character
Starting with generatiing with less than 1000 charater. In following cells it will start from 1000 then 300 then 500 as length.

300 one is definitly giving better results cause it follows up with said and gives meaningful sentence and words mostly.

500 one is better than 1000 by making sentences but its not best like 300. But still i think its arguably good.

In [9]:
generate_text(bigger_model)

Seed:
" erself down to her usual height.

it was so long since she had been anything near the right size, th "


Generated:
e was a little sharp came an excellent to the thore the had been anything that she was a little tharp little children she had been anything to say the thing had something to do it as the could, and was going to suite a little birds and the hatter, and the whole party and said to alice, and she went on again: “i’ve shen a curious thing as ‘ou say them and the best tay that ‘ 
“you can’t have cone to the same thing as a groiog of the sea,” said the duchess: “and the moral of that is—‘beres if you could know you were listenses to be a letter, and the moral of that is—‘beres and such a thing as all the mock turtle. and the mock turtle sather so the thought to herself, “i wonder what i was going to say ‘what _ dueat with the bread-and-butter—”

“what did they did you to tell me,” said the duchess: “and the moral of that is—‘beres if you could know you were listenses an

In [10]:
generate_text(bigger_model, length=300)

Seed:
" d the king, looking round the
court with a smile. there was a dead silence.

“it’s a pun!” the king  "


Generated:
said to alice as she could.

“the way a good deal tize to your pardon!” said the dodo.

“well, i’ve think of mothing of the sea, the dodo such a thing as all the conk tuch a catcus-race and a little bottle had so say the thing some of them with the door, she had not gone to say the door and the whit

Done


In [11]:
generate_text(bigger_model, length=500)

Seed:
"  must be what he
did with the tarts, you know—”

“but, it goes on ‘_they all returned from him to yo "


Generated:
u to go at all a little birds and the best and the bread-and-butter, and the three gardeners in the thingle before the had not gone of the court, and the white rabbit was an angry to the hatter, and the whole party and said to alice, and she went on again: “i’ve shen a curious thing as ‘ou say them and the best tay that ‘ 
“you can’t have cone to the same thing as a groiog of the sea,” said the duchess: “and the moral of that is—‘beres if you could know you were listenses to be a letter, and the

Done


#2. Cleaning text more
As result we left with 29 characters. 27 letter and one line break and one whitespace. Lets train with this for 20 epoch

After 20 epochs experimented with couple text generations. But they started repeat themself so experiment is failed. More detail in next lines.

In [18]:
# load ascii text and covert to lowercase
filename = "wonderland_removed_punc.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: {}".format(n_chars))
print("Total Vocab: {}".format(n_vocab))
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: {}".format(n_patterns))
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalise the inputs to be in the range 0..1
X = X / float(n_vocab)
# one hot encode the outputs
y = to_categorical(dataY)

Total Characters: 135816
Total Vocab: 29
Total Patterns: 135716


In [19]:
chars

['\n',
 ' ',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'ù']

In [21]:
# Redefine the model
bigger_model = Sequential()
bigger_model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
bigger_model.add(Dropout(0.2))
bigger_model.add(LSTM(256, return_sequences=True))
bigger_model.add(Dropout(0.2))
bigger_model.add(LSTM(256))
bigger_model.add(Dropout(0.2))
bigger_model.add(Dense(y.shape[1], activation='softmax'))
bigger_model.compile(loss='categorical_crossentropy', optimizer='adam')
bigger_model.summary()
bigger_filepath="weights-improvement-puncremoved-{epoch:02d}-{loss:.4f}-bigger.keras"
bigger_checkpoint = ModelCheckpoint(bigger_filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
bigger_callbacks_list = [bigger_checkpoint]
# load the network weights
int_to_char = dict((i, c) for i, c in enumerate(chars))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                        │ (None, 100, 256)            │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 100, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 100, 256)            │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 100, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 256)                 │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 29)                  │           7,453 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,322,269 (5.04 MB)

 Trainable params: 1,322,269 (5.04 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# fit the model
bigger_model.fit(X, y, epochs=20, batch_size=64, callbacks=bigger_callbacks_list)

Epoch 1/20
2118/2121 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.8461
Epoch 1: loss improved from inf to 2.69175, saving model to weights-improvement-puncremoved-01-2.6917-bigger.keras
2121/2121 ━━━━━━━━━━━━━━━━━━━━ 31s 14ms/step - loss: 2.8458
Epoch 2/20
2118/2121 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.2443
Epoch 2: loss improved from 2.69175 to 2.16520, saving model to weights-improvement-puncremoved-02-2.1652-bigger.keras
2121/2121 ━━━━━━━━━━━━━━━━━━━━ 29s 14ms/step - loss: 2.2442
Epoch 3/20
2120/2121 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.9734
Epoch 3: loss improved from 2.16520 to 1.93786, saving model to weights-improvement-puncremoved-03-1.9379-bigger.keras
2121/2121 ━━━━━━━━━━━━━━━━━━━━ 29s 14ms/step - loss: 1.9733
Epoch 4/20
2120/2121 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.8245
Epoch 4: loss improved from 1.93786 to 1.80644, saving model to weights-improvement-puncremoved-04-1.8064-bigger.keras
2121/2121 ━━━━━━━━━━━━━━━━━━━━ 29s 14ms/step - loss: 1.8244
Epoch 5/

In [32]:
# you can try with this to use my trained model. file is in submission
filename = "weights-improvement-puncremoved-20-1.2399-bigger.keras"
bigger_model.load_weights(filename)
bigger_model.compile(loss='categorical_crossentropy', optimizer='adam')

#Conclusion
Here 3 experiment made with 500, 300, and 100 length. It literally repeated itself really quickly. Seed is not really good like alice but not very bad also. 100 one is the best seed I think. But results are really bad. I also tried with some other checkpoint to make sure its not specific to this checkpoint but it also failed like this one. checkpont .keras file is also shared in submission.

In [33]:
generate_text(bigger_model,length=500)

Seed:
" ill the roof bearmind that loose slateoh its coming down
heads below a loud crashnow who did thatit  "


Generated:
was the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle 

Done


In [34]:
generate_text(bigger_model,length=300)

Seed:
" id the eaglet i dont know the meaning of half
those long words and whats more i dont believe you do  "


Generated:
so the soom and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turtle and the 

Done


In [35]:
generate_text(bigger_model,length=100)

Seed:
"  lessons
thought alice  i might as well be at school at once however she got
up and began to repeat  "


Generated:
it was the mock turtle and the mock turtle and the mock turtle and the mock turtle and the mock turt

Done


# 3. Lets Try a new source file.
I decided to try with "The adventures of pinocchio". file is attached also in submission

In [2]:
# load ascii text and covert to lowercase
filename = "pinocchio.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: {}".format(n_chars))
print("Total Vocab: {}".format(n_vocab))
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: {}".format(n_patterns))
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalise the inputs to be in the range 0..1
X = X / float(n_vocab)
# one hot encode the outputs
y = to_categorical(dataY)


Total Characters: 213505
Total Vocab: 53
Total Patterns: 213405


In [46]:
# Redefine the model
pinocchio_model = Sequential()
pinocchio_model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
pinocchio_model.add(Dropout(0.2))
pinocchio_model.add(LSTM(256, return_sequences=True))
pinocchio_model.add(Dropout(0.2))
pinocchio_model.add(LSTM(256))
pinocchio_model.add(Dropout(0.2))
pinocchio_model.add(Dense(y.shape[1], activation='softmax'))
pinocchio_model.compile(loss='categorical_crossentropy', optimizer='adam')
pinocchio_model.summary()
pinocchio_filepath="weights-pinocchio-{epoch:02d}-{loss:.4f}-bigger.keras"
pinocchio_checkpoint = ModelCheckpoint(pinocchio_filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
pinocchio_callbacks_list = [pinocchio_checkpoint]
# load the network weights
int_to_char = dict((i, c) for i, c in enumerate(chars))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_12 (LSTM)                       │ (None, 100, 256)            │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 100, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_13 (LSTM)                       │ (None, 100, 256)            │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 100, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_14 (LSTM)                       │ (None, 256)                 │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 53)                  │          13,621 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,328,437 (5.07 MB)

 Trainable params: 1,328,437 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
# fit the model
pinocchio_model.fit(X, y, epochs=20, batch_size=64, callbacks=pinocchio_callbacks_list)

Epoch 1/20
3333/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.8999
Epoch 1: loss improved from inf to 2.68564, saving model to weights-pinocchio-01-2.6856-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 48s 14ms/step - loss: 2.8997
Epoch 2/20
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.2627
Epoch 2: loss improved from 2.68564 to 2.19334, saving model to weights-pinocchio-02-2.1933-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 45s 14ms/step - loss: 2.2627
Epoch 3/20
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.0113
Epoch 3: loss improved from 2.19334 to 1.98283, saving model to weights-pinocchio-03-1.9828-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 46s 14ms/step - loss: 2.0113
Epoch 4/20
3333/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.8720
Epoch 4: loss improved from 1.98283 to 1.85586, saving model to weights-pinocchio-04-1.8559-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 46s 14ms/step - loss: 1.8720
Epoch 5/20
3334/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1

In [ ]:
# you can try with this to use my trained model. file is in submission
filename = "weights-pinocchio-20-1.3316-bigger.keras"
pinocchio_model.load_weights(filename)
pinocchio_model .compile(loss='categorical_crossentropy', optimizer='adam')

In [40]:
generate_text(bigger_model,length=300)

Seed:
" een swallowed by the terrible shark, which,
for the last few days, has been bringing terror to these "


Generated:
 words. the fox and the cat, and the poor marionette was about to see the sea and said to him:

“what is it?”

“bnd what is it?”

“i am sorry for you. the fairy will be so seally a boy, i have not a bookey and i have not a bookey and i have not a bookey and i have not a bookey and i have not a booke

Done


In [42]:
generate_text(bigger_model,length=500)

Seed:
"  the opening of all prison doors.

“if the others go, i go, too,” said pinocchio to the jailer.

“no "


Generated:
t i  i am sorry for you!”

“what is it?”

“bnd what is it?”

“i am sorry for you. the fairy will be so seally a boy, i have not a bookey and i have not a bookey and i have not a bookey and i have not a bookey and i have not a bookey and i have not a bookey and i have not a bookey and i have not a bookey and i have not a bookey and i have not a bookey and i have not a bookey and i have not a bookey and i have not a bookey and i have not a bookey and i have not a bookey and i have not a bookey and

Done


Again it tend to repeat itself lets try with higher epochs if it can prevent it.

# 4. More epochs.
I will add 20 more epoch.

Result is below, it solved the issue with length of 500. So as the epochs increase and loss is reduce it becomes more reliable model. 20 epoch is low for this structure but, it takes time to train.

In [43]:
pinocchio_model.fit(X, y, epochs=20, batch_size=64, callbacks=bigger_callbacks_list)

Epoch 1/20
3334/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.3048
Epoch 1: loss improved from 1.33156 to 1.32127, saving model to weights-pinocchio-01-1.3213-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 46s 14ms/step - loss: 1.3048
Epoch 2/20
3334/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.2939
Epoch 2: loss improved from 1.32127 to 1.31042, saving model to weights-pinocchio-02-1.3104-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 45s 14ms/step - loss: 1.2940
Epoch 3/20
3332/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.2843
Epoch 3: loss improved from 1.31042 to 1.29824, saving model to weights-pinocchio-03-1.2982-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 45s 14ms/step - loss: 1.2843
Epoch 4/20
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.2813
Epoch 4: loss improved from 1.29824 to 1.29121, saving model to weights-pinocchio-04-1.2912-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 45s 14ms/step - loss: 1.2813
Epoch 5/20
3332/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - los

In [ ]:
# you can try with this to use my trained model. file is in submission
filename = "weights-pinocchio-20-1.1967-bigger.keras"
pinocchio_model.load_weights(filename)
pinocchio_model .compile(loss='categorical_crossentropy', optimizer='adam')

In [44]:
generate_text(bigger_model,length=500)

Seed:
" n a marionette’s life there’s always a but which is apt to
spoil everything.




chapter 30

pinocch "


Generated:
io said, the fairy with azure hair and the streets, and the shark said to him:

“do you want to go to school, i should not eat you, the shark, i have seen him. and i have been a boy, and the fairy will be so long and the shark’s stomach and the books of the soom. and the shark said to himself:
“the way is the same time that we are an any on the sea, and the marionette said to himself:

“the shark and the fairy will be so long at i am seachers and i have no more than a beautiful little voice and 

Done


It finally resolved now there is no issue of repeating itself. We can increate epochs more. Its also seen that loss is reducing so there are still space to train. But I want to experiment batch size,dropout and new layers now.
First changing dropout to 3.
# 5.Change dropout to 0.3

Dropout made it to learn way more slower we can see it in loss.

In [3]:
# Redefine the model
pinocchio_model_dropout = Sequential()
pinocchio_model_dropout.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
pinocchio_model_dropout.add(Dropout(0.3))
pinocchio_model_dropout.add(LSTM(256, return_sequences=True))
pinocchio_model_dropout.add(Dropout(0.3))
pinocchio_model_dropout.add(LSTM(256))
pinocchio_model_dropout.add(Dropout(0.3))
pinocchio_model_dropout.add(Dense(y.shape[1], activation='softmax'))
pinocchio_model_dropout.compile(loss='categorical_crossentropy', optimizer='adam')
pinocchio_model_dropout.summary()
pinocchio_filepath="weights-pinocchio-dropout-{epoch:02d}-{loss:.4f}-bigger.keras"
pinocchio_checkpoint = ModelCheckpoint(pinocchio_filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
pinocchio_callbacks_list = [pinocchio_checkpoint]
# load the network weights
int_to_char = dict((i, c) for i, c in enumerate(chars))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 100, 256)            │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100, 256)            │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 100, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 256)                 │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 53)                  │          13,621 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,328,437 (5.07 MB)

 Trainable params: 1,328,437 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
pinocchio_model_dropout.fit(X, y, epochs=40, batch_size=64, callbacks=pinocchio_callbacks_list)

Epoch 1/40
3334/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 3.0164
Epoch 1: loss improved from inf to 2.87110, saving model to weights-pinocchio-dropout-01-2.8711-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 51s 14ms/step - loss: 3.0163
Epoch 2/40
3332/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.5233
Epoch 2: loss improved from 2.87110 to 2.44326, saving model to weights-pinocchio-dropout-02-2.4433-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 45s 14ms/step - loss: 2.5232
Epoch 3/40
3333/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.2524
Epoch 3: loss improved from 2.44326 to 2.20978, saving model to weights-pinocchio-dropout-03-2.2098-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 46s 14ms/step - loss: 2.2524
Epoch 4/40
3332/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2.0960
Epoch 4: loss improved from 2.20978 to 2.06947, saving model to weights-pinocchio-dropout-04-2.0695-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 45s 14ms/step - loss: 2.0960
Epoch 5/40
3333/3335 ━━━━━━━━━━━

In [ ]:
# you can try with this to use my trained model. file is in submission
filename = "weights-pinocchio-dropout-40-1.3769-bigger.keras"
pinocchio_model.load_weights(filename)
pinocchio_model .compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
generate_text(pinocchio_model_dropout,length=300)

Seed:
" t he
burst an artery and died on the spot.

pinocchio freed himself from his awkward position and on "


Generated:
e of the shore was so strnng and the marionette was as beginning to the fox and the marionette was as beginning to the fox and the marionette was as beginning to the fox and the marionette was as beginning to the fox and the marionette was as beginning to the fox and the marionette was as beginning 

Done


In [9]:
generate_text(pinocchio_model_dropout,length=500)

Seed:
" ge wave came and the boat disappeared.

they waited and waited for it, but it was gone.

“poor man!” "


Generated:
 said the marionette, “and what is the shark and the shark will be a boy ”

“what is the shark and the shark will be a boy, i want to go to tchool. i want to go to tchool. i want to go to tchool. i want to go to tchool. i want to go to tchool. i want to go to tchool. i want to go to tchool. i want to go to tchool. i want to go to tchool. i want to go to tchool. i want to go to tchool. i want to go to tchool. i want to go to tchool. i want to go to tchool. i want to go to tchool. i want to go to 

Done


It started to repeat again, I was expecting this because of the losses it needs more training.
# 6. Batch size to 32.
I want to experiment new batch size of 32 here. Batch size makes it even more hard to train. But worth to experiment. 20 more epochs with 32 batch size.

After training 5 epochs losses were not good. I didnt complete 20. To understand if its because of batch size below I will reload model and train 5 epochs with 64 batch size to compare losses.

And it went good with looses. So making high dropout with high batch size is not good trade off. The results of losses can be seen below.

In [13]:
pinocchio_model_dropout.fit(X, y, epochs=20, batch_size=32, callbacks=pinocchio_callbacks_list)

Epoch 1/20
6669/6669 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.4391
Epoch 1: loss did not improve from 1.37694
6669/6669 ━━━━━━━━━━━━━━━━━━━━ 92s 14ms/step - loss: 1.4391
Epoch 2/20
6669/6669 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.4205
Epoch 2: loss did not improve from 1.37694
6669/6669 ━━━━━━━━━━━━━━━━━━━━ 92s 14ms/step - loss: 1.4205
Epoch 3/20
6668/6669 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.4139
Epoch 3: loss did not improve from 1.37694
6669/6669 ━━━━━━━━━━━━━━━━━━━━ 92s 14ms/step - loss: 1.4139
Epoch 4/20
6669/6669 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.4130
Epoch 4: loss did not improve from 1.37694
6669/6669 ━━━━━━━━━━━━━━━━━━━━ 92s 14ms/step - loss: 1.4130
Epoch 5/20
6667/6669 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.4122
Epoch 5: loss did not improve from 1.37694
6669/6669 ━━━━━━━━━━━━━━━━━━━━ 91s 14ms/step - loss: 1.4122
Epoch 6/20
2453/6669 ━━━━━━━━━━━━━━━━━━━━ 57s 14ms/step - loss: 1.3957

KeyboardInterrupt: 

In [16]:
# you can try with this to use my trained model. file is in submission
filename = "weights-pinocchio-dropout-40-1.3769-bigger.keras"
pinocchio_model_dropout.load_weights(filename)
pinocchio_model_dropout .compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
pinocchio_model_dropout.fit(X, y, epochs=5, batch_size=64, callbacks=pinocchio_callbacks_list)

Epoch 1/5
3332/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.3688
Epoch 1: loss improved from 1.37694 to 1.37595, saving model to weights-pinocchio-dropout-01-1.3759-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 48s 14ms/step - loss: 1.3688
Epoch 2/5
3334/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.3569
Epoch 2: loss improved from 1.37595 to 1.36965, saving model to weights-pinocchio-dropout-02-1.3696-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 46s 14ms/step - loss: 1.3569
Epoch 3/5
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.3579
Epoch 3: loss improved from 1.36965 to 1.36411, saving model to weights-pinocchio-dropout-03-1.3641-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 46s 14ms/step - loss: 1.3579
Epoch 4/5
3332/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.3539
Epoch 4: loss improved from 1.36411 to 1.35792, saving model to weights-pinocchio-dropout-04-1.3579-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 46s 14ms/step - loss: 1.3539
Epoch 5/5
3335/3335 ━━━━━━━━━━━━

Loses are improved in 64 batch. So its clear that 32 batch size is not a good choice here
# 7. More layers
I think more layer will not make it good. Model is already complex enough to "understand" book. But here is experiment

Result: I was expecting to be bad. But not this bad. You can see it in results. I am not sure why seed is so bad. but definitly adding a new layer didnt solve anything.

In [21]:
# Redefine the model
pinocchio_model_layered = Sequential()
pinocchio_model_layered.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
pinocchio_model_layered.add(Dropout(0.2))
pinocchio_model_layered.add(LSTM(256, return_sequences=True))
pinocchio_model_layered.add(Dropout(0.2))
pinocchio_model_layered.add(LSTM(256, return_sequences=True))
pinocchio_model_layered.add(Dropout(0.2))
pinocchio_model_layered.add(LSTM(256))
pinocchio_model_layered.add(Dropout(0.2))
pinocchio_model_layered.add(Dense(y.shape[1], activation='softmax'))
pinocchio_model_layered.compile(loss='categorical_crossentropy', optimizer='adam')
pinocchio_model_layered.summary()
pinocchio_filepath="weights-pinocchio-layered-{epoch:02d}-{loss:.4f}-bigger.keras"
pinocchio_checkpoint = ModelCheckpoint(pinocchio_filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
pinocchio_callbacks_list = [pinocchio_checkpoint]
# load the network weights
int_to_char = dict((i, c) for i, c in enumerate(chars))

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_7 (LSTM)                        │ (None, 100, 256)            │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 100, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 100, 256)            │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 100, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 100, 256)            │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 100, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_10 (LSTM)                       │ (None, 256)                 │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 53)                  │          13,621 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,853,749 (7.07 MB)

 Trainable params: 1,853,749 (7.07 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
pinocchio_model_layered.fit(X, y, epochs=20, batch_size=64, callbacks=pinocchio_callbacks_list)

Epoch 1/20
3333/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 3.0699
Epoch 1: loss improved from inf to 3.02285, saving model to weights-pinocchio-layered-01-3.0228-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 63s 18ms/step - loss: 3.0698
Epoch 2/20
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 2.6464
Epoch 2: loss improved from 3.02285 to 2.51687, saving model to weights-pinocchio-layered-02-2.5169-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 61s 18ms/step - loss: 2.6464
Epoch 3/20
3334/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 2.2378
Epoch 3: loss improved from 2.51687 to 2.17880, saving model to weights-pinocchio-layered-03-2.1788-bigger.keras
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 61s 18ms/step - loss: 2.2378
Epoch 4/20
3334/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 2.2774
Epoch 4: loss did not improve from 2.17880
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 60s 18ms/step - loss: 2.2775
Epoch 5/20
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 2.1682
Epoch 5: loss improved from 2.17

In [24]:
# epoch 7 is best reloading it.
# you can try with this to use my trained model. file is in submission
filename = "weights-pinocchio-layered-07-1.8683-bigger.keras"
pinocchio_model_layered.load_weights(filename)
pinocchio_model_layered .compile(loss='categorical_crossentropy', optimizer='adam')

In [25]:
generate_text(pinocchio_model_layered,length=300)

Seed:
" d)65 a2. d)f 3
d)8a a9 19 6.a 7. _.. 366 13c. a2.7 ) 8.d :)3? 90 *99a_ .c.?f
,2?3_a7)_

)8- _2. d.8a "


Generated:
e t 
“ao darll w 
“ao darll w 
“ao darll t 
“ao darll t 
“ao l 
“ao darlllll t 
“ao l 
“ao cholllll t

“ao l 
“ao l 
“ao chollllll t

“ao l 
“ao l 
“ao l 
“ao cholllllll t

“ao l 
“ao l 
“ao cholllll t

“ao l 
“ao l 
“ao cholllll t

“ao l 
“ao l 
“ao chollllll t

“ao l 
“ao l 
“ao chollllll t

“ao l

Done


In [26]:
generate_text(pinocchio_model_layered,length=500)

Seed:
"  d.?. 89 )?,2._ 6.0a )8-
)66 a2. :6)f.?_ .e,.:a a2. 5381 a2. ;b..8 )8- )63,. d.?. 38 ,b_a9-f
)8- b8- "


Generated:
t 
“ao darll w 
“ao darll t 
“ao darll t 
“ao darll t 
“ao darllllll t 
“ao l 
“ao l 
“ao cholllll t

“ao l 
“ao l 
“ao cholllllll t

“ao l 
“ao l 
“ao l 
“ao chollllll t

“ao l 
“ao l 
“ao cholllll t

“ao l 
“ao l 
“ao chollllll t

“ao l 
“ao l 
“ao chollllll t

“ao l 
“ao l 
“ao chollllll t

“ao l 
“ao l 
“ao l 
“ao cholllllll t

“ao l 
“ao l 
“ao l 
“ao cholllllll t

“ao l 
“ao l 
“ao cholllll t

“ao l 
“ao l 
“ao cholllll t

“ao l 
“ao l 
“ao chollllll t

“ao l 
“ao l 
“ao chollllll t

“ao l

Done


This is the end. All given experiments have been done. I tried to walkthotough everyhting i have done in experiments includeing my expectations and what actually received. As a conclusion trade-offs really matter and I think i understood some of the trade-offs happeing.

Thanks. Hope you like it .